In [1]:
from lxml import etree
import pandas as pd
import sys 
from ast import literal_eval
from itertools import combinations, product
sys.path.append('../types/')
import pickle
import json
import numpy as np

In [2]:
with open("./parsed/clone_frags.pkl", "rb") as f:
    clone_frags = pickle.load(f)

In [3]:
def aggregate_clone_frags(group):
    return [{
        'repoName': frag['repoName'],
        'userName': frag['userName'],
        'fileName': frag['fileName'], 
        'startline': frag['startline'], 
        'endline': frag['endline'], 
        'pcid': frag['pcid']
    } for _, frag in group.iterrows()]

# group and aggregate by bucketId, userName
acrossRepos_df = clone_frags.groupby(by=['bucketId', 'similarity']).apply(aggregate_clone_frags).reset_index(name='repoObjects').sort_values(by=['bucketId'], ascending=True)


/tmp/ipykernel_9842/3329900840.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  acrossRepos_df = clone_frags.groupby(by=['bucketId', 'similarity']).apply(aggregate_clone_frags).reset_index(name='repoObjects').sort_values(by=['bucketId'], ascending=True)


In [4]:
def find_across_repo_pairs(repoObjects):
    all_pairs = list(combinations(repoObjects, 2))
    # Filter pairs to keep only those from different repositories**
    across_repo_pairs = [pair for pair in all_pairs if pair[0]['repoName'] != pair[1]['repoName']]
    return across_repo_pairs

# Apply this function to each group to get acrossRepoPairs and add them as a new column
acrossRepos_df['acrossRepoPairs'] = acrossRepos_df['repoObjects'].apply(find_across_repo_pairs)


In [5]:
acrossRepos_df

,bucketId,similarity,repoObjects,acrossRepoPairs
0,1,74,"[{'repoName': 'Flask-PyTorch-Chatbot', 'userNa...","[({'repoName': 'Flask-PyTorch-Chatbot', 'userN..."
1,2,70,"[{'repoName': 'NSRR-PyTorch', 'userName': 'IMA...","[({'repoName': 'NSRR-PyTorch', 'userName': 'IM..."
2,3,71,"[{'repoName': 'torchtext_lite', 'userName': 'r...",[]
3,4,84,"[{'repoName': 'torchtext_lite', 'userName': 'r...",[]
4,5,81,[{'repoName': 'IMDB-Sentiment-Analysis-using-P...,[({'repoName': 'IMDB-Sentiment-Analysis-using-...
...,...,...,...,...
3322,3323,75,"[{'repoName': 'PyTorch-UNet', 'userName': 'mcg...",[]
3323,3324,80,[{'repoName': 'PyTorch-CNN-for-RUL-Prediction'...,[]
3324,3325,93,"[{'repoName': 'DeepVis-PredDiff-PyTorch', 'use...",[]
3325,3326,77,"[{'repoName': 'ProgGAN-PyTorch', 'userName': '...",[]


In [6]:
# filter out acrossRepoPairs that are []
acrossRepos_df = acrossRepos_df[acrossRepos_df['acrossRepoPairs'].apply(lambda x: len(x) > 0)]

In [7]:
def findIfSameUser(pairList):
    for pair in pairList:
        if pair[0]['userName'] != pair[1]['userName']:
            return False
    return True

acrossRepos_df['sameUser'] = acrossRepos_df['acrossRepoPairs'].apply(findIfSameUser)

/tmp/ipykernel_9842/4103917434.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acrossRepos_df['sameUser'] = acrossRepos_df['acrossRepoPairs'].apply(findIfSameUser)


In [8]:
acrossRepos_df['acrossRepoPairsCount'] = acrossRepos_df['acrossRepoPairs'].apply(lambda x: len(x))

/tmp/ipykernel_9842/1325004521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acrossRepos_df['acrossRepoPairsCount'] = acrossRepos_df['acrossRepoPairs'].apply(lambda x: len(x))


In [10]:
acrossRepos_df

,bucketId,similarity,repoObjects,acrossRepoPairs,sameUser,acrossRepoPairsCount
0,1,74,"[{'repoName': 'Flask-PyTorch-Chatbot', 'userNa...","[({'repoName': 'Flask-PyTorch-Chatbot', 'userN...",False,1
1,2,70,"[{'repoName': 'NSRR-PyTorch', 'userName': 'IMA...","[({'repoName': 'NSRR-PyTorch', 'userName': 'IM...",False,66
4,5,81,[{'repoName': 'IMDB-Sentiment-Analysis-using-P...,[({'repoName': 'IMDB-Sentiment-Analysis-using-...,False,1
5,6,98,"[{'repoName': 'ssdetection', 'userName': 'Dete...","[({'repoName': 'ssdetection', 'userName': 'Det...",True,1
6,7,80,"[{'repoName': 'ssdetection', 'userName': 'Dete...","[({'repoName': 'ssdetection', 'userName': 'Det...",True,4
...,...,...,...,...,...,...
3140,3141,83,[{'repoName': 'FlowNet2-PyTorch-EasyToUse-Wrap...,[({'repoName': 'FlowNet2-PyTorch-EasyToUse-Wra...,False,1
3197,3198,70,"[{'repoName': 'ReID.PyTorch', 'userName': 'zhu...","[({'repoName': 'ReID.PyTorch', 'userName': 'zh...",False,1
3270,3271,78,"[{'repoName': 'SSE-PT-torch', 'userName': 'hsu...","[({'repoName': 'SSE-PT-torch', 'userName': 'hs...",False,1
3302,3303,100,"[{'repoName': 'Deep_Learning_PyTorch', 'userNa...","[({'repoName': 'Deep_Learning_PyTorch', 'userN...",False,1


In [9]:
acrossRepos_df.to_pickle("./parsed/acrossRepos_df.pkl")